![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 5: Building Agents

**⏱️ Time:** 60 minutes

## 🎯 Learning Objectives

By the end of this module, you will:

1. **Understand** LangGraph fundamentals (nodes, edges, state)
2. **Build** memory tools that LLMs can call
3. **Implement** a complete course advisor agent
4. **Compare** passive vs active memory management

---

## 📚 Part 1: LangGraph Fundamentals (15 min)

### What is LangGraph?

**LangGraph** is a framework for building stateful, multi-step AI agents.

Key concepts:
- **Nodes**: Functions that process state
- **Edges**: Connections between nodes
- **State**: Data that flows through the graph

### Why LangGraph for Agents?

| Feature | Benefit |
|---------|--------|
| **State Management** | Track conversation, tools, decisions |
| **Conditional Routing** | Different paths based on LLM decisions |
| **Tool Integration** | Easy to add tools the LLM can call |
| **Debugging** | Visualize agent flow |

In [1]:
# Setup
import os
import sys
import json
from pathlib import Path
from typing import Annotated, TypedDict

repo_root = Path.cwd().parent
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()  # Try current dir first
load_dotenv(repo_root / ".env")  # Then try parent

print("✅ Setup complete!")

✅ Setup complete!


In [2]:
# Define Agent State
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """State that flows through the agent graph."""
    messages: Annotated[list, add_messages]  # Conversation history
    user_id: str                              # Student identifier
    session_id: str                           # Session identifier

print("AgentState defined with:")
print("  - messages: Conversation history")
print("  - user_id: Student identifier")
print("  - session_id: Session identifier")

AgentState defined with:
  - messages: Conversation history
  - user_id: Student identifier
  - session_id: Session identifier


### Simple Graph Example

```
START → process_input → generate_response → END
```

In [3]:
# Simple graph example
from langchain_core.messages import HumanMessage, AIMessage

def process_input(state: AgentState) -> AgentState:
    """Process user input."""
    print(f"Processing: {state['messages'][-1].content}")
    return state

def generate_response(state: AgentState) -> AgentState:
    """Generate a response."""
    response = AIMessage(content="Hello! I'm your course advisor.")
    return {"messages": [response]}

# Build the graph
simple_graph = StateGraph(AgentState)
simple_graph.add_node("process", process_input)
simple_graph.add_node("respond", generate_response)
simple_graph.add_edge("process", "respond")
simple_graph.add_edge("respond", END)
simple_graph.set_entry_point("process")

# Compile
simple_agent = simple_graph.compile()
print("✅ Simple graph compiled!")

✅ Simple graph compiled!


In [4]:
# Run the simple graph
result = simple_agent.invoke({
    "messages": [HumanMessage(content="Hello!")],
    "user_id": "test_user",
    "session_id": "test_session"
})

print(f"\nResponse: {result['messages'][-1].content}")

Processing: Hello!

Response: Hello! I'm your course advisor.


---

## 📚 Part 2: Memory Tools (20 min)

### Passive vs Active Memory

**Module 3 (Passive):** You explicitly call memory functions
```python
working_memory = await memory_client.get_working_memory(...)
facts = await memory_client.search_long_term_memory(...)
```

**Module 4 (Active):** LLM decides when to use memory tools
```python
@tool
def store_memory(text: str):
    """Store important information."""

@tool  
def search_memories(query: str):
    """Search for relevant facts."""
```

**Key Insight:** Tools let the LLM actively decide when to use memory!

In [5]:
# Define memory tools (demo mode - in-memory storage)
# In production, these would use Agent Memory Server
from langchain_core.tools import tool
from typing import Dict, List

# Simple in-memory stores for demo
memory_store: Dict[str, List[str]] = {}

@tool
def store_memory(text: str, user_id: str) -> str:
    """Store important information about the student in long-term memory.
    Use this when the student shares preferences, goals, or important facts.
    
    Args:
        text: The information to store
        user_id: The student's identifier
    """
    if user_id not in memory_store:
        memory_store[user_id] = []
    memory_store[user_id].append(text)
    return f"Stored: {text}"

@tool
def search_memories(query: str, user_id: str) -> str:
    """Search long-term memory for relevant facts about the student.
    Use this to recall preferences, history, or goals.
    
    Args:
        query: What to search for
        user_id: The student's identifier
    """
    facts = memory_store.get(user_id, [])
    if not facts:
        return "No relevant memories found."
    # Simple keyword matching (production uses embeddings)
    query_words = set(query.lower().split())
    relevant = [f for f in facts if any(w in f.lower() for w in query_words)]
    if not relevant:
        return "\n".join([f"- {f}" for f in facts[:3]])
    return "\n".join([f"- {f}" for f in relevant[:3]])

print("✅ Memory tools defined!")
print("  - store_memory: Save important facts")
print("  - search_memories: Find relevant facts")

✅ Memory tools defined!
  - store_memory: Save important facts
  - search_memories: Find relevant facts


In [6]:
# Define course search tool (demo mode - uses sample data)

# Sample course data
SAMPLE_COURSES = [
    {"code": "CS301", "title": "Machine Learning", "level": "intermediate", "credits": 4},
    {"code": "CS401", "title": "Deep Learning", "level": "advanced", "credits": 4},
    {"code": "CS402", "title": "Natural Language Processing", "level": "advanced", "credits": 3},
    {"code": "CS201", "title": "Data Structures", "level": "beginner", "credits": 3},
    {"code": "MATH201", "title": "Linear Algebra", "level": "intermediate", "credits": 3},
]

@tool
def search_courses(query: str) -> str:
    """Search for courses matching the query.
    Returns course summaries with details for top matches.
    
    Args:
        query: What kind of courses to search for
    """
    # Simple keyword matching (production uses vector search)
    query_words = set(query.lower().split())
    results = []
    for course in SAMPLE_COURSES:
        course_text = f"{course['title']} {course['level']}".lower()
        if any(w in course_text for w in query_words):
            results.append(course)
    
    if not results:
        results = SAMPLE_COURSES[:3]  # Return top 3 if no match
    
    output = "Available Courses:\n"
    for c in results[:5]:
        output += f"  • {c['code']}: {c['title']} ({c['level']}, {c['credits']} credits)\n"
    return output

print("✅ Course search tool defined!")

✅ Course search tool defined!


---

## 📚 Part 3: Complete Agent (20 min)

### Agent Architecture

```
START → agent_node → should_continue? → tool_node → agent_node → ... → END
                  ↘                                           ↗
                    → END (if no tools needed)
```

In [7]:
# Build the complete agent
# Note: This requires OpenAI API key. We'll show the structure and simulate.
from langgraph.prebuilt import ToolNode

# Initialize tools list
tools = [store_memory, search_memories, search_courses]

# System prompt
SYSTEM_PROMPT = """You are a helpful university course advisor.

You have access to these tools:
- search_courses: Find courses matching a query
- search_memories: Recall facts about the student
- store_memory: Save important student information

Guidelines:
1. Search memories first to understand the student's context
2. Store important preferences or goals the student shares
3. Search courses to find relevant recommendations
4. Explain your reasoning clearly
"""

print("✅ Tools and system prompt configured!")
print(f"\nAvailable tools: {[t.name for t in tools]}")

✅ Tools and system prompt configured!

Available tools: ['store_memory', 'search_memories', 'search_courses']


In [8]:
# Define agent node (simulated for demo)
from langchain_core.messages import SystemMessage

def agent_node(state: AgentState) -> AgentState:
    """The main agent node that decides what to do."""
    # In production: response = llm_with_tools.invoke(messages)
    # For demo, we simulate the agent's decision
    last_msg = state["messages"][-1].content.lower()
    
    if "machine learning" in last_msg or "interested" in last_msg:
        # Simulate storing preference and searching courses
        response = AIMessage(content="I've noted your interest in machine learning! Let me search for relevant courses.")
    elif "recommend" in last_msg:
        response = AIMessage(content="Based on your interest in machine learning, I recommend CS301: Machine Learning and CS401: Deep Learning.")
    else:
        response = AIMessage(content="How can I help you find courses today?")
    
    return {"messages": [response]}

# Define routing function
def should_continue(state: AgentState) -> str:
    """Decide whether to continue with tools or end."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

print("✅ Agent node and routing defined!")

✅ Agent node and routing defined!


In [9]:
# Build the graph
from langgraph.graph import StateGraph, END

# Create tool node
tool_node = ToolNode(tools)

# Build graph
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)

# Add edges
graph.set_entry_point("agent")
graph.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "end": END}
)
graph.add_edge("tools", "agent")

# Compile
course_advisor = graph.compile()
print("✅ Course advisor agent compiled!")

✅ Course advisor agent compiled!


In [10]:
# Test the agent (demo mode)
import uuid

user_id = "workshop_student_001"
session_id = str(uuid.uuid4())

# First interaction - share preferences
result = course_advisor.invoke({
    "messages": [HumanMessage(content="Hi! I'm interested in machine learning and prefer online courses.")],
    "user_id": user_id,
    "session_id": session_id
})

print("Agent Response:")
print("="*60)
print(result["messages"][-1].content)

Agent Response:
I've noted your interest in machine learning! Let me search for relevant courses.


In [11]:
# Second interaction - ask for recommendations
result = course_advisor.invoke({
    "messages": [HumanMessage(content="What courses would you recommend for me?")],
    "user_id": user_id,
    "session_id": session_id
})

print("Agent Response:")
print("="*60)
print(result["messages"][-1].content)

Agent Response:
Based on your interest in machine learning, I recommend CS301: Machine Learning and CS401: Deep Learning.


### Tool Execution Demo

Let's see how the tools work directly:

In [12]:
# Demonstrate tools directly
print("Tool Execution Demo:")
print("="*60)

# Store a memory
result1 = store_memory.invoke({"text": "Interested in machine learning", "user_id": user_id})
print(f"store_memory: {result1}")

# Search memories
result2 = search_memories.invoke({"query": "machine learning", "user_id": user_id})
print(f"\nsearch_memories: {result2}")

# Search courses
result3 = search_courses.invoke({"query": "machine learning"})
print(f"\nsearch_courses:\n{result3}")

Tool Execution Demo:
store_memory: Stored: Interested in machine learning

search_memories: - Interested in machine learning

search_courses:
Available Courses:
  • CS301: Machine Learning (intermediate, 4 credits)
  • CS401: Deep Learning (advanced, 4 credits)



---

## 🎯 Key Takeaways

1. **LangGraph** provides structure for multi-step agents
2. **State** flows through nodes, accumulating information
3. **Tools** let the LLM actively decide when to use capabilities
4. **Memory tools** enable dynamic context engineering
5. **Conditional routing** creates flexible agent behavior

---

## ➡️ Next Module

In **Module 6: Capstone Comparison**, you'll see:
- Side-by-side comparison of Stage 4 vs Stage 6 agents
- Key metrics and trade-offs
- Production patterns and next steps